# 📞 Phony Voice AI Quick Start Tutorial

This interactive notebook will guide you through setting up and making your first AI phone call with Phony.

## 🎯 What You'll Learn

- How to configure Phony for voice AI calls
- Making outbound AI calls to humans
- Handling inbound calls from humans
- Monitoring calls with the dashboard
- Using the REST API

## 📋 Prerequisites

Before running this notebook, ensure you have:
- Docker and Docker Compose installed
- OpenAI API key with Realtime API access
- Twilio account with a phone number
- Phony services running (`docker-compose up -d`)

## 🔧 Environment Setup

First, let's verify our environment and check that Phony is running correctly.

In [ ]:
import requests
import json
import os
from datetime import datetime
import time

# Configuration
BASE_URL = "http://localhost:24187"
API_KEY = os.getenv("PHONY_API_KEY", "demo-key")  # Replace with actual API key

# Headers for API requests
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

print("🔧 Phony QuickStart Configuration")
print(f"Base URL: {BASE_URL}")
print(f"API Key: {API_KEY[:12]}..." if API_KEY != "demo-key" else "API Key: demo-key (update for production)")
print(f"Timestamp: {datetime.now()}")

## ✅ Health Check

Let's verify that Phony is running and healthy:

In [ ]:
# Check system health
try:
    response = requests.get(f"{BASE_URL}/healthz")
    health_data = response.json()
    
    print("🟢 System Health Check")
    print(f"Status: {health_data['status']}")
    print(f"Uptime: {health_data['uptime']} seconds")
    print(f"Active Calls: {health_data['activeCalls']}")
    
    if health_data['status'] == 'ok':
        print("✅ Phony is running correctly!")
    else:
        print("❌ System is not healthy")
        
except Exception as e:
    print(f"❌ Error connecting to Phony: {e}")
    print("🔧 Make sure Docker services are running: docker-compose up -d")

## 🤖 Agent Management

Let's explore the AI agents available in your Phony installation:

In [ ]:
# List available agents
try:
    response = requests.get(f"{BASE_URL}/agents/", headers=headers)
    
    if response.status_code == 200:
        agents_data = response.json()
        agents = agents_data.get('agents', [])
        
        print(f"🤖 Available AI Agents ({len(agents)} total)")
        print("=" * 50)
        
        for i, agent in enumerate(agents, 1):
            print(f"{i}. {agent['name']}")
            print(f"   ID: {agent['id']}")
            print(f"   Voice: {agent.get('voice', 'alloy')}")
            print(f"   Status: {'🟢 Active' if agent.get('is_active') else '🔴 Inactive'}")
            print(f"   Phone Numbers: {', '.join(agent.get('phone_numbers', []))}")
            print(f"   Created: {agent.get('created_at', 'Unknown')}")
            print()
            
    else:
        print(f"❌ Error fetching agents: {response.status_code}")
        print("💡 This might be expected if multi-tenancy is not set up")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Agent management may not be available in single-tenant mode")

## 🎭 AI Personalities Demo

Phony comes with 5 pre-configured AI personalities. Let's explore what each one does:

In [ ]:
# Define the AI personalities
personalities = {
    1: {
        "name": "Professional Assistant",
        "voice": "alloy",
        "description": "Business helper and support specialist",
        "use_cases": ["Customer service", "Technical support", "Business inquiries"],
        "greeting": "Hello! This is your professional assistant. How may I help you today?"
    },
    2: {
        "name": "Customer Service Rep", 
        "voice": "echo",
        "description": "Technical support and issue resolution",
        "use_cases": ["Technical troubleshooting", "Product support", "Account issues"],
        "greeting": "Hi there! I'm here to help resolve any technical issues you might have."
    },
    3: {
        "name": "Appointment Scheduler",
        "voice": "nova", 
        "description": "Booking coordination and scheduling",
        "use_cases": ["Appointment booking", "Calendar management", "Scheduling conflicts"],
        "greeting": "Good day! I'm your scheduling assistant. Let's find the perfect time for your appointment."
    },
    4: {
        "name": "Information Hotline",
        "voice": "fable",
        "description": "General information and FAQ responses", 
        "use_cases": ["General inquiries", "Information lookup", "FAQ responses"],
        "greeting": "Welcome to our information hotline! What information can I help you find today?"
    },
    5: {
        "name": "Survey Conductor",
        "voice": "shimmer",
        "description": "Feedback collection and market research",
        "use_cases": ["Customer surveys", "Feedback collection", "Market research"],
        "greeting": "Hi! I'd love to get your feedback. This will just take a couple of minutes."
    }
}

print("🎭 AI Personalities Overview")
print("=" * 60)

for pid, personality in personalities.items():
    print(f"\n{pid}. {personality['name']} (Voice: {personality['voice']})")
    print(f"   📝 {personality['description']}")
    print(f"   🎯 Use Cases: {', '.join(personality['use_cases'])}")
    print(f"   💬 Sample Greeting: \"{personality['greeting']}\"")

print("\n📞 To test these personalities, call: +1 (857) 816-7225")

## 📱 Making API Calls

Let's demonstrate how to interact with Phony programmatically using the REST API:

In [ ]:
# Example: Create a new agent (if multi-tenant mode is enabled)
def create_sample_agent():
    agent_data = {
        "name": "Demo Assistant",
        "system_prompt": "You are a helpful demo assistant for the Phony Voice AI system. Keep responses brief and friendly.",
        "voice": "alloy",
        "model": "gpt-4o-realtime-preview",
        "greeting_message": "Hello! This is a demo of the Phony Voice AI system. How can I assist you?",
        "phone_numbers": ["+15551234567"],  # Example number
        "is_active": True
    }
    
    try:
        response = requests.post(f"{BASE_URL}/agents/", headers=headers, json=agent_data)
        
        if response.status_code == 201:
            agent = response.json()
            print("✅ Sample agent created successfully!")
            print(f"Agent ID: {agent['id']}")
            print(f"Name: {agent['name']}")
            return agent
        else:
            print(f"❌ Error creating agent: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Try to create agent (this might fail if multi-tenancy is not configured)
print("🤖 Testing Agent Creation")
print("Note: This may fail if running in single-tenant mode")
print("-" * 50)

sample_agent = create_sample_agent()

if not sample_agent:
    print("\n💡 Agent creation failed - this is normal for single-tenant installations")
    print("   You can still use the pre-configured demo number: +1 (857) 816-7225")

## 📊 Dashboard Integration

The Phony dashboard provides real-time call monitoring. Let's generate a dashboard URL:

In [ ]:
# Generate dashboard URLs
dashboard_base = f"{BASE_URL}/dashboard/"
sample_call_sid = "CA1234567890abcdef1234567890abcdef"

print("📊 Phony Dashboard Access")
print("=" * 40)
print(f"\n🏠 Main Dashboard: {dashboard_base}")
print(f"📞 Call Monitor: {dashboard_base}index.html?callSid={sample_call_sid}")
print(f"📈 Health Status: {BASE_URL}/healthz")

print("\n💡 Dashboard Features:")
features = [
    "Real-time call transcripts",
    "Supervisor intervention controls",
    "DTMF keypad for sending digits", 
    "Call transfer and termination",
    "Live event stream monitoring",
    "Audio quality metrics"
]

for feature in features:
    print(f"   ✅ {feature}")

# Test dashboard accessibility
try:
    response = requests.get(dashboard_base)
    if response.status_code == 200:
        print("\n✅ Dashboard is accessible!")
    else:
        print(f"\n❌ Dashboard access issue: {response.status_code}")
except Exception as e:
    print(f"\n❌ Dashboard connection error: {e}")

## 🧪 Testing Voice AI Features

Let's explore the key features you can test with Phony:

In [ ]:
# Testing scenarios
test_scenarios = {
    "outbound": {
        "title": "AI Calls Human (Outbound)",
        "command": "docker-compose --profile human run --rm human-demo",
        "steps": [
            "Select option 1 (AI calls human)",
            "Confirm consent: yes", 
            "Enter target phone number",
            "Choose scenario (1-4)"
        ],
        "scenarios": [
            "1. Customer Service Inquiry",
            "2. Survey/Feedback Request",
            "3. Appointment Scheduling", 
            "4. Friendly Check-in"
        ]
    },
    "inbound": {
        "title": "Human Calls AI (Inbound)",
        "command": "docker-compose --profile human run --rm human-demo",
        "steps": [
            "Select option 2 (Human calls AI)",
            "Choose personality (1-5)",
            "Call +1 (857) 816-7225"
        ],
        "personalities": [
            "1. Professional Assistant",
            "2. Customer Service Rep", 
            "3. Appointment Scheduler",
            "4. Information Hotline",
            "5. Survey Conductor"
        ]
    }
}

print("🧪 Voice AI Testing Guide")
print("=" * 50)

for test_type, config in test_scenarios.items():
    print(f"\n📞 {config['title']}")
    print(f"Command: {config['command']}")
    print("\nSteps:")
    for i, step in enumerate(config['steps'], 1):
        print(f"   {i}. {step}")
    
    if 'scenarios' in config:
        print("\nAvailable Scenarios:")
        for scenario in config['scenarios']:
            print(f"   {scenario}")
    
    if 'personalities' in config:
        print("\nAvailable Personalities:")
        for personality in config['personalities']:
            print(f"   {personality}")

print("\n🔧 Additional Testing Commands:")
test_commands = [
    "docker-compose run --rm demo python3 scripts/test_human_demo_suite.py",
    "docker-compose run --rm demo python3 scripts/test_edge_cases.py", 
    "docker-compose run --rm demo python3 scripts/test_websocket_fix.py"
]

for cmd in test_commands:
    print(f"   {cmd}")

## 📈 Performance Monitoring

Let's check some basic performance metrics:

In [ ]:
# Performance monitoring
def check_performance():
    metrics = {}
    
    # Health check with timing
    start_time = time.time()
    try:
        response = requests.get(f"{BASE_URL}/healthz", timeout=5)
        response_time = (time.time() - start_time) * 1000
        
        if response.status_code == 200:
            health_data = response.json()
            metrics = {
                "status": health_data['status'],
                "uptime": health_data['uptime'],
                "active_calls": health_data['activeCalls'],
                "response_time_ms": round(response_time, 2)
            }
        else:
            metrics["error"] = f"HTTP {response.status_code}"
            
    except Exception as e:
        metrics["error"] = str(e)
    
    return metrics

print("📈 System Performance Check")
print("=" * 40)

perf_data = check_performance()

if "error" not in perf_data:
    print(f"✅ Status: {perf_data['status']}")
    print(f"⏱️  Response Time: {perf_data['response_time_ms']} ms")
    print(f"🕐 Uptime: {perf_data['uptime']} seconds")
    print(f"📞 Active Calls: {perf_data['active_calls']}")
    
    # Performance assessment
    if perf_data['response_time_ms'] < 100:
        print("🟢 Excellent response time")
    elif perf_data['response_time_ms'] < 500:
        print("🟡 Good response time")
    else:
        print("🔴 Slow response time - check system load")
        
else:
    print(f"❌ Performance check failed: {perf_data['error']}")

print("\n💡 Performance Tips:")
tips = [
    "Monitor response times during calls",
    "Check Docker container resource usage",
    "Ensure stable internet connection for WebSockets",
    "Monitor OpenAI API rate limits"
]

for tip in tips:
    print(f"   • {tip}")

## 🎯 Next Steps

Congratulations! You've completed the Phony Voice AI Quick Start tutorial. Here's what to explore next:

In [ ]:
# Next steps and resources
next_steps = {
    "🎭 Customize AI Personality": {
        "description": "Modify system prompts and voices",
        "file": "Edit SYSTEM_PROMPT in .env",
        "docs": "tutorials/custom-agent"
    },
    "🏗️ Production Deployment": {
        "description": "Deploy to production environment", 
        "file": "docker-compose.prod.yml",
        "docs": "tutorials/deployment-guide"
    },
    "🔌 API Integration": {
        "description": "Integrate with existing systems",
        "file": "Check api/ documentation",
        "docs": "api/endpoints"
    },
    "📊 Advanced Monitoring": {
        "description": "Set up analytics and alerts",
        "file": "Dashboard configuration",
        "docs": "ui-guide/dashboard-overview" 
    },
    "👥 Multi-Tenant Setup": {
        "description": "Configure multiple agents/tenants",
        "file": "Multi-tenant configuration",
        "docs": "features/multi-tenant-architecture"
    }
}

print("🎯 Recommended Next Steps")
print("=" * 50)

for title, info in next_steps.items():
    print(f"\n{title}")
    print(f"   📝 {info['description']}")
    print(f"   📁 {info['file']}")
    print(f"   📚 Documentation: {info['docs']}")

print("\n📞 Testing Resources:")
print("   • Live Demo Number: +1 (857) 816-7225")
print("   • Dashboard: http://localhost:24187/dashboard/")
print("   • API Docs: http://localhost:24187/docs")
print("   • Health Check: http://localhost:24187/healthz")

print("\n🆘 Support:")
print("   • GitHub Issues: https://github.com/sackio/phony/issues")
print("   • Documentation: Full guides in docs/ directory")
print("   • Community: Contribute and get help")

print("\n✅ Tutorial Complete! Happy voice AI building! 🎉")